<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [6]</a>'.</span>

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
import pandas as pd
import os
import shutil

PYSPARK_PYTHON = os.getenv("PYSPARK_PYTHON") 
PYSPARK_DRIVER_PYTHON = os.getenv("PYSPARK_DRIVER_PYTHON")

In [2]:
import pyspark
from delta import *
import json

# Load the configuration JSON file
with open('/usr/local/spark/conf/spark-defaults.json', 'r') as f:
    config = json.load(f)

# Initialize the Spark session builder
builder = pyspark.sql.SparkSession.builder.appName("MyApp1").config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog").config("spark.pyspark.python", PYSPARK_PYTHON)\
    .config("spark.pyspark.driver.python", PYSPARK_DRIVER_PYTHON)

# Read the packages from the text file
packages = []
with open('/usr/local/spark/conf/packages.txt', 'r') as file:
    # Read each line and strip newlines or extra spaces
    packages = [line.strip() for line in file if line.strip()]

# # Add packages to the Spark session configuration
builder.config("spark.jars.packages", ",".join(packages))

# Apply the configurations from the JSON file to the Spark session
for key, value in config.items():
    builder.config(key, value)

In [3]:
# Spark will automatically use the master specified in spark-defaults.conf
spark = configure_spark_with_delta_pip(builder).getOrCreate()

spark

In [4]:
dfconfig=pd.read_csv('/home/jovyan/Notebooks/configpath.csv')
dfpath=spark.createDataFrame(dfconfig)

In [5]:
trgt_path_processed = dfpath.filter(col("DataFeedName") == "Category_Delta_Path").select('Path').collect()[0][0]
trgt_path_csv = dfpath.filter(col("DataFeedName") == "Category_CSV_Path").select('Path').collect()[0][0]
source_path = dfpath.filter(col("DataFeedName") == "FactFile").select('Path').collect()[0][0]
v_sheet_name =dfpath.filter(col("DataFeedName") == "sheet_name").select('Path').collect()[0][0]

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [6]:
# Read the Excel file (use Spark-Excel library)
df = pd.read_excel(source_path, sheet_name = v_sheet_name)
df = spark.createDataFrame(df)
df= df.withColumn("Date", to_date(df["Date"],"yyyy-MM-dd"))

OSError: [Errno 35] Resource deadlock avoided

In [ ]:
df.createOrReplaceTempView("vw_src")
query = "select distinct Category from vw_src"
print(query)
df_src = spark.sql(query)
df_src.show()

In [ ]:
df_output = \
df_src.withColumn("categorysk",xxhash64("category"))\
        .withColumn("UpdateTimeStamp", date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss").cast("timestamp"))\
            .withColumn("RowSK",xxhash64(concat_ws("|", *[col(c) for c in df_src.columns])))

In [ ]:
df_output.createOrReplaceTempView("vw_source")

In [ ]:
df_output.createOrReplaceTempView("vw_source")
if DeltaTable.isDeltaTable(spark, trgt_path_processed):
    column_name = df_output.columns
    set_clause = ", ".join([f"target.{i} = source.{i}" for i in column_name])
    insert_clause=",".join(column_name)
    insert_values=",".join([f"source.{i}" for i in column_name])
    query = f"""MERGE INTO delta.`{trgt_path_processed}` AS target 
            USING vw_source AS source 
            ON target.categorysk = source.categorysk 
            AND target.RowSK <> source.RowSK 
            WHEN MATCHED THEN UPDATE SET {set_clause}
            WHEN NOT MATCHED THEN INSERT ({insert_clause}) VALUES ({insert_values})"""
else :
    query=f"CREATE TABLE delta.`{trgt_path_processed}` USING DELTA AS SELECT * FROM vw_source"
print(query)
spark.sql(query)

In [ ]:
# Save the DataFrame to a CSV file
spark.read.format("delta").load(trgt_path_processed).coalesce(1).write.format("csv").option("header","true").mode("overwrite").save(trgt_path_csv)

In [ ]:
trgt_copy_path = trgt_path_csv + "processed.csv"
files=os.listdir(trgt_path_csv)
selected_files = [file for file in files if file.startswith('part-00') and file.endswith('.csv')]
file=trgt_path_csv + selected_files[0]
print(selected_files)
shutil.copy(file, trgt_copy_path)

In [ ]:
delete_log = [file for file in files if "processed.csv" != file ]
for file in delete_log :
    os.remove(trgt_path_csv + file)
    print(f"removed {trgt_path_csv + file}")

In [ ]:
spark.stop()